In [1]:
import pandas as pd
import numpy as np
import requests
import sqlite3
from plyer import notification 

##### Envio de notificação de ERRO

In [2]:
def alerta(nivel, url, etapa, erro=""):

    msg = f"Falha no carregamento da base {url} na etapa {etapa}.\n{erro}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )

##### 1 - Realiza get na api cidade

In [3]:
def get_base_localidades():
    url = "https://brasilapi.com.br/api/cptec/v1/cidade"

    try:
        response = requests.get(url)
        data_json = response.json()
    except Exception as e:
        alerta(nivel = 3, 
               url = url, 
               etapa = (f"REQUEST, {response}"), 
               erro=e)


    df_localidades = pd.DataFrame(data_json)

    df_localidades = df_localidades.rename(columns={'nome':'cidade','id':'id_cidade'})

    return df_localidades

##### 1.1 - Print das localidades

In [4]:
get_base_localidades()

,cidade,id_cidade,estado
0,São Benedito,4750,CE
1,São Benedito do Rio Preto,4751,MA
2,São Benedito do Sul,4752,PE
3,São Bento,4753,MA
4,São Bento,4754,PB
5,São Bento Abade,4755,MG
6,São Bento de Pombal,4756,PB
7,São Bento do Norte,4757,RN
8,São Bento do Sapucaí,4758,SP
9,São Bento do Sul,4759,SC


##### 2 - Realiza get na api previsao do tempo atraves do id_cidade do item 1

In [5]:
#Gera a previsao do tempo para cada cidade do item 1 atraves do id_cidade
def get_previsao_tempo():
    cidades = get_base_localidades()
    list_previsao = []

    #Utiliza o id_cidade para gerar a previsao do tempo para cada cidade
    for id_cityCode in cidades.id_cidade:
        try:
            url = (f'https://brasilapi.com.br/api/cptec/v1/clima/previsao/' + str(id_cityCode)) 
            response_previsao = requests.get(url)
            data_jason = response_previsao.json()
            list_previsao.append (data_jason)
        except Exception as e:
            alerta(nivel = 3, 
                url = url, 
                etapa = (f"REQUEST, {response_previsao}"), 
                erro=e)
            break

    
    return list_previsao

##### 2.1 - Print da previsão do tempo do dia + 1

In [7]:
get_previsao_tempo()

[{'cidade': 'São Benedito',
  'estado': 'CE',
  'atualizado_em': '2024-04-25',
  'clima': [{'data': '2024-04-25',
    'condicao': 'pn',
    'condicao_desc': 'Parcialmente Nublado',
    'min': 21,
    'max': 30,
    'indice_uv': 11}]},
 {'cidade': 'São Benedito do Rio Preto',
  'estado': 'MA',
  'atualizado_em': '2024-04-25',
  'clima': [{'data': '2024-04-25',
    'condicao': 'ci',
    'condicao_desc': 'Chuvas Isoladas',
    'min': 24,
    'max': 33,
    'indice_uv': 11}]},
 {'cidade': 'São Benedito do Sul',
  'estado': 'PE',
  'atualizado_em': '2024-04-25',
  'clima': [{'data': '2024-04-25',
    'condicao': 'pn',
    'condicao_desc': 'Parcialmente Nublado',
    'min': 22,
    'max': 31,
    'indice_uv': 10}]},
 {'cidade': 'São Bento',
  'estado': 'MA',
  'atualizado_em': '2024-04-25',
  'clima': [{'data': '2024-04-25',
    'condicao': 'c',
    'condicao_desc': 'Chuva',
    'min': 24,
    'max': 31,
    'indice_uv': 12}]},
 {'cidade': 'São Bento',
  'estado': 'PB',
  'atualizado_em': '2

##### 3 - Extrai do json do item 2 as informacoes do clima, temperatura minima e temperatura maxima, após gera Dataframe com essas informacoes

In [8]:
#Funcao para criar Dataframe contendo clima, temperatura minima e temperatura maxima 
def separa_condicoes_tempo():
    clima = get_previsao_tempo()
    condicoes = []
    temp_min = []
    temp_max = []
    data = []
    for dados in clima:
        for json in dados.get('clima'):
            data.append(json.get('data'))
            condicoes.append(json.get('condicao_desc'))
            temp_min.append(json.get('min'))
            temp_max.append(json.get('max'))
  
    
    df_clima = pd.DataFrame(condicoes, columns=['clima'])
    df_temp_min = pd.DataFrame(temp_min, columns=['minima'])
    df_temp_max = pd.DataFrame(temp_max, columns=['maxima'])

    concat_clima = pd.concat([df_clima,df_temp_min,df_temp_max], axis=1)
    return concat_clima

##### 3.1 - Print as condições do tempo

In [9]:
separa_condicoes_tempo()

,clima,minima,maxima
0,Parcialmente Nublado,21,30
1,Chuvas Isoladas,24,33
2,Parcialmente Nublado,22,31
3,Chuva,24,31
4,Parcialmente Nublado,24,35
5,Parcialmente Nublado,17,30
6,Parcialmente Nublado,24,34
7,Parcialmente Nublado,23,35
8,Parcialmente Nublado,15,26
9,Chuva,18,19


##### 4 - Gera DataFrame com as informacoes da cidade, id_cidade, estado, clima, temparatura minima e temperatura maxima

In [10]:
df_previsaoTempo = pd.concat([get_base_localidades(),separa_condicoes_tempo()], axis=1)
df_previsaoTempo

,cidade,id_cidade,estado,clima,minima,maxima
0,São Benedito,4750,CE,Parcialmente Nublado,21,30
1,São Benedito do Rio Preto,4751,MA,Chuvas Isoladas,24,33
2,São Benedito do Sul,4752,PE,Parcialmente Nublado,22,31
3,São Bento,4753,MA,Chuva,24,31
4,São Bento,4754,PB,Parcialmente Nublado,24,35
5,São Bento Abade,4755,MG,Parcialmente Nublado,17,30
6,São Bento de Pombal,4756,PB,Parcialmente Nublado,24,34
7,São Bento do Norte,4757,RN,Parcialmente Nublado,23,35
8,São Bento do Sapucaí,4758,SP,Parcialmente Nublado,15,26
9,São Bento do Sul,4759,SC,Chuva,18,19


##### 5 - Gera a previsao do tempo para cada cidade do item 1 atraves do id_cidade, para os próximos 6 dias

In [11]:

def get_previsao_tempo_v2():
    cidades = get_base_localidades()
    list_previsao = []

            #Utiliza o id_cidade para gerar a previsao do tempo para cada cidade
    for id_cityCode in cidades.id_cidade:
            try:
                url = (f'https://brasilapi.com.br/api/cptec/v1/clima/previsao/' + str(id_cityCode) +'/6') 
                response_previsao = requests.get(url)
                data_jason = response_previsao.json()
                list_previsao.append (data_jason)
            except Exception as e:
                alerta(nivel = 3, 
                        url = url, 
                        etapa = (f"REQUEST, {response_previsao}"), 
                        erro=e)
                break
    df = pd.DataFrame(list_previsao)
    clima = df.clima
    return clima

##### 5.1 - Print no formato json da data, condições do tempo, temperatura minima e temperatura máxima dos próximos 5 dias do item 1

In [13]:
get_previsao_tempo_v2()

0     [{'data': '2024-04-25', 'condicao': 'pn', 'con...
1     [{'data': '2024-04-25', 'condicao': 'ci', 'con...
2     [{'data': '2024-04-25', 'condicao': 'pn', 'con...
3     [{'data': '2024-04-25', 'condicao': 'c', 'cond...
4     [{'data': '2024-04-25', 'condicao': 'pn', 'con...
5     [{'data': '2024-04-25', 'condicao': 'pn', 'con...
6     [{'data': '2024-04-25', 'condicao': 'pn', 'con...
7     [{'data': '2024-04-25', 'condicao': 'pn', 'con...
8     [{'data': '2024-04-25', 'condicao': 'pn', 'con...
9     [{'data': '2024-04-25', 'condicao': 'c', 'cond...
10    [{'data': '2024-04-25', 'condicao': 'pn', 'con...
11    [{'data': '2024-04-25', 'condicao': 'pn', 'con...
12    [{'data': '2024-04-25', 'condicao': 'pn', 'con...
13    [{'data': '2024-04-25', 'condicao': 'c', 'cond...
14    [{'data': '2024-04-25', 'condicao': 'c', 'cond...
15    [{'data': '2024-04-25', 'condicao': 'c', 'cond...
16    [{'data': '2024-04-25', 'condicao': 'c', 'cond...
17    [{'data': '2024-04-25', 'condicao': 'c', '

##### 6 - Gera pandas DataFrame da data, condições do tempo, temperatura minima e temperatura máxima dos próximos 5 dias do item 5

In [16]:
def separa_clima_cinco_dias():
    clima = get_previsao_tempo_v2()
    
    list_data = []
    lista1 =[]
    lista2 =[]
    lista3 =[]
    lista4 =[]
    lista5 =[]
    lista6 =[]
    list_condicoes = []
    clima1 = []
    clima2 = []
    clima3 = []
    clima4 = []
    clima5 = []
    clima6 = []
    list_temp_min = []
    min1 = []
    min2 = []
    min3 = []
    min4 = []
    min5 = []
    min6 = []
    list_temp_max = []
    max1 = []
    max2 = []
    max3 = []
    max4 = []
    max5 = []
    max6 = []

    for climas in clima:
        for dados in climas:
            list_data.append(dados.get('data'))
            list_condicoes.append(dados.get('condicao_desc'))
            list_temp_min.append(dados.get('min'))
            list_temp_max.append(dados.get('max'))

    i=0
    while i < len(list_data):
        i = i + 1
        lista1.append(list_data[i])
        df_data1 = pd.DataFrame(lista1, columns=['data1'])
        clima1.append(list_condicoes[i])
        df_clima1 = pd.DataFrame(clima1, columns=['clima1']) 
        min1.append(list_temp_min[i])
        df_min1 = pd.DataFrame(min1, columns=['min1']) 
        max1.append(list_temp_max[i])
        df_max1 = pd.DataFrame(max1, columns=['max1'])
        i = i + 1
        
        lista2.append(list_data[i])
        df_data2 = pd.DataFrame(lista2, columns=['data2'])
        clima2.append(list_condicoes[i])
        df_clima2 = pd.DataFrame(clima2, columns=['clima2'])
        min2.append(list_temp_min[i])
        df_min2 = pd.DataFrame(min2, columns=['min2']) 
        max2.append(list_temp_max[i])
        df_max2 = pd.DataFrame(max2, columns=['max2'])
        i = i + 1
        
        lista3.append(list_data[i])
        df_data3 = pd.DataFrame(lista3, columns=['data3'])
        clima3.append(list_condicoes[i])
        df_clima3 = pd.DataFrame(clima3, columns=['clima3'])
        min3.append(list_temp_min[i])
        df_min3 = pd.DataFrame(min3, columns=['min3']) 
        max3.append(list_temp_max[i])
        df_max3 = pd.DataFrame(max3, columns=['max3'])
        i = i + 1
        
        lista4.append(list_data[i])
        df_data4 = pd.DataFrame(lista4, columns=['data4'])
        clima4.append(list_condicoes[i])
        df_clima4 = pd.DataFrame(clima4, columns=['clima4'])
        min4.append(list_temp_min[i])
        df_min4 = pd.DataFrame(min4, columns=['min4']) 
        max4.append(list_temp_max[i])
        df_max4 = pd.DataFrame(max4, columns=['max4'])
        i = i + 1

        lista5.append(list_data[i])
        df_data5 = pd.DataFrame(lista5, columns=['data5'])
        clima5.append(list_condicoes[i])
        df_clima5 = pd.DataFrame(clima5, columns=['clima5'])
        min5.append(list_temp_min[i])
        df_min5 = pd.DataFrame(min5, columns=['min5']) 
        max5.append(list_temp_max[i])
        df_max5 = pd.DataFrame(max5, columns=['max5'])
        i = i + 1   

        data_concat = pd.concat([df_data1, df_clima1, df_min1, df_max1,
                                df_data2, df_clima2, df_min2, df_max2,
                                df_data3, df_clima3, df_min3, df_max3,
                                df_data4, df_clima4, df_min4, df_max4,
                                df_data5, df_clima5, df_min5, df_max5], axis=1)
                                #df_data6, df_clima6, df_min6, df_max6
    clima_concat = data_concat

    return clima_concat

##### 6.1 - Print do intem 6

In [17]:
separa_clima_cinco_dias()

,data1,clima1,min1,max1,data2,clima2,min2,max2,data3,clima3,min3,max3,data4,clima4,min4,max4,data5,clima5,min5,max5
0,2024-04-26,Parcialmente Nublado,21,30,2024-04-27,Parcialmente Nublado,21,29,2024-04-28,Chuvas Isoladas,22,28,2024-04-29,Chuva,21,28,2024-04-30,Chuva,21,28
1,2024-04-26,Parcialmente Nublado,24,35,2024-04-27,Chuva,24,32,2024-04-28,Chuva,24,31,2024-04-29,Chuva,23,30,2024-04-30,Chuva,23,31
2,2024-04-26,Chuvas Isoladas,23,31,2024-04-27,Chuva,24,30,2024-04-28,Chuva,24,30,2024-04-29,Chuvas Isoladas,22,27,2024-04-30,Chuva,22,29
3,2024-04-26,Parcialmente Nublado,23,32,2024-04-27,Chuva,24,34,2024-04-28,Chuva,24,29,2024-04-29,Chuva,24,28,2024-04-30,Chuva,23,31
4,2024-04-26,Parcialmente Nublado,22,37,2024-04-27,Chuvas Isoladas,24,36,2024-04-28,Parcialmente Nublado,24,35,2024-04-29,Parcialmente Nublado,23,36,2024-04-30,Chuvas Isoladas,23,36
5,2024-04-26,Parcialmente Nublado,17,28,2024-04-27,Parcialmente Nublado,17,29,2024-04-28,Parcialmente Nublado,19,30,2024-04-29,Parcialmente Nublado,20,31,2024-04-30,Parcialmente Nublado,21,32
6,2024-04-26,Parcialmente Nublado,22,36,2024-04-27,Chuvas Isoladas,23,36,2024-04-28,Chuvas Isoladas,23,35,2024-04-29,Chuvas Isoladas,22,35,2024-04-30,Chuvas Isoladas,23,36
7,2024-04-26,Parcialmente Nublado,23,34,2024-04-27,Chuvas Isoladas,24,32,2024-04-28,Parcialmente Nublado,24,34,2024-04-29,Chuva,24,33,2024-04-30,Chuva,24,31
8,2024-04-26,Parcialmente Nublado,16,25,2024-04-27,Parcialmente Nublado,15,26,2024-04-28,Parcialmente Nublado,17,27,2024-04-29,Parcialmente Nublado,18,28,2024-04-30,Parcialmente Nublado,17,29
9,2024-04-26,Parcialmente Nublado,17,25,2024-04-27,Parcialmente Nublado,17,30,2024-04-28,Parcialmente Nublado,18,30,2024-04-29,Chuva,19,31,2024-04-30,Chuva,18,28


##### 7 - Conatena:
#####                get_base_localidades() - cidade, id_cidade, estado +
#####                separa_condicoes_tempo() - clima do dia atual
#####                separa_clima_cinco_dias() - clima dos próximos 6 dias

In [18]:
df_previsaoTempo = pd.concat([get_base_localidades(),separa_condicoes_tempo(),separa_clima_cinco_dias()], axis=1)
df_previsaoTempo.index

RangeIndex(start=0, stop=60, step=1)

##### 8 - Conecta ao banco de dados Sqlite

In [19]:
def tabelas_bd():

    conn = sqlite3.connect('coderhouse.db')

    # Executar uma consulta que retorna as informações do esquema do banco de dados
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema


##### 9 - Salva no banco de dados

In [20]:
def salva_bd(df, nome_tabela):

    conn = sqlite3.connect('coderhouse_prev_temp.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True

##### 10 - Carrega em banco

In [21]:
def carrega_bd(nome_tabela):

    conn = sqlite3.connect('coderhouse_prev_temp.db')

    # Executar uma consulta SELECT na tabela 'produtos' e converter em um DataFrame
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)

    conn.close()

    return df

##### 11 - Salva 0 resultado da DataFrame na tabel em banco

In [22]:
salva_bd(df_previsaoTempo,'previsao_tempo')

True

##### 12 - Carrega no banco

In [23]:
select_banco = carrega_bd('previsao_tempo')
select_banco

,cidade,id_cidade,estado,clima,minima,maxima,data1,clima1,min1,max1,...,min3,max3,data4,clima4,min4,max4,data5,clima5,min5,max5
0,São Benedito,4750,CE,Parcialmente Nublado,21,30,2024-04-26,Parcialmente Nublado,21,30,...,22,28,2024-04-29,Chuva,21,28,2024-04-30,Chuva,21,28
1,São Benedito do Rio Preto,4751,MA,Chuvas Isoladas,24,33,2024-04-26,Parcialmente Nublado,24,35,...,24,31,2024-04-29,Chuva,23,30,2024-04-30,Chuva,23,31
2,São Benedito do Sul,4752,PE,Parcialmente Nublado,22,31,2024-04-26,Chuvas Isoladas,23,31,...,24,30,2024-04-29,Chuvas Isoladas,22,27,2024-04-30,Chuva,22,29
3,São Bento,4753,MA,Chuva,24,31,2024-04-26,Parcialmente Nublado,23,32,...,24,29,2024-04-29,Chuva,24,28,2024-04-30,Chuva,23,31
4,São Bento,4754,PB,Parcialmente Nublado,24,35,2024-04-26,Parcialmente Nublado,22,37,...,24,35,2024-04-29,Parcialmente Nublado,23,36,2024-04-30,Chuvas Isoladas,23,36
5,São Bento Abade,4755,MG,Parcialmente Nublado,17,30,2024-04-26,Parcialmente Nublado,17,28,...,19,30,2024-04-29,Parcialmente Nublado,20,31,2024-04-30,Parcialmente Nublado,21,32
6,São Bento de Pombal,4756,PB,Parcialmente Nublado,24,34,2024-04-26,Parcialmente Nublado,22,36,...,23,35,2024-04-29,Chuvas Isoladas,22,35,2024-04-30,Chuvas Isoladas,23,36
7,São Bento do Norte,4757,RN,Parcialmente Nublado,23,35,2024-04-26,Parcialmente Nublado,23,34,...,24,34,2024-04-29,Chuva,24,33,2024-04-30,Chuva,24,31
8,São Bento do Sapucaí,4758,SP,Parcialmente Nublado,15,26,2024-04-26,Parcialmente Nublado,16,25,...,17,27,2024-04-29,Parcialmente Nublado,18,28,2024-04-30,Parcialmente Nublado,17,29
9,São Bento do Sul,4759,SC,Chuva,18,19,2024-04-26,Parcialmente Nublado,17,25,...,18,30,2024-04-29,Chuva,19,31,2024-04-30,Chuva,18,28
